## Library Importation

In [49]:
# import necessary libraries
import json
import copy


## Read Images

In [50]:
# Read colored image
with open("data/prof_barry.json") as infile:
    col_img = json.load(infile)

# Read floating point gray image
with open("data/prof_barry_gray.json") as infile:
    gray_img = json.load(infile)

# Read quantized gray image
with open("data/prof_barry_gray_quant.json") as infile:
    gray_quant_img = json.load(infile)

## Definition of the RGB Matrix and Grayscale Vectors

In [51]:
# shape of colored image is: (533, 799, 3)
# shape of grayscale image is: (533, 799)
# shape of quantized grayscale image is: (533, 799)

# Get rgb matrix and grayscale vectors
rgb_matrix  = [col_img[0][0], col_img[0][2], col_img[0][3]] # RGB matrix
gray_matrix = [gray_img[0][0], gray_img[0][2], gray_img[0][3]] # floating-point grayscale values
gray_quant_matrix = [gray_quant_img[0][0], gray_quant_img[0][2], gray_quant_img[0][3]] # integer grayscale values

## Finding the Inverse of a 3x3 Matrix Using Gauss-Jordan and Cofactors

In [52]:
def inv_gauss_jordan(mat: list) -> list:
    """ 
    This function uses the Gauss-jordan elimination method
    to find the inverse of a matrix.

    Arguments
    ---------
    mat: matrix to be inverted (list-of-lists)

    Results
    -------
    inv: inverse of matrix
    """
    
    # define the identity matrix
    id_mat = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    
    # define the augmented matrix
    aug_mat = [row_mat + id_mat[i] for i, row_mat in enumerate(mat)]

    # Gauss-Jordan elimination
    size = 3  # We are working with 3x3 matrices
    
    for i in range(size):
        pivot = aug_mat[i][i]
        
        # Check for a non-zero pivot
        if pivot == 0:
            # Find a row with a non-zero pivot and swap it with the current row
            for j in range(i + 1, size):
                if aug_mat[j][i] != 0:
                    aug_mat[i], aug_mat[j] = aug_mat[j], aug_mat[i]
                    break
            else:
                # If no row with a non-zero pivot is found, the matrix is singular and cannot be inverted.
                # Return an empty list to indicate this.
                return []
            pivot = aug_mat[i][i]
            # raise ValueError("Matrix is singular, cannot compute inverse.")
        
        # Scale the current row to make the pivot equal to 1
        for j in range(size * 2):
            aug_mat[i][j] /= pivot
        
        # Eliminate other rows
        for k in range(size):
            if k == i:
                continue
            ratio = aug_mat[k][i]
            for j in range(size * 2):
                aug_mat[k][j] -= ratio * aug_mat[i][j]

    # Extract the inverse matrix from the augmented matrix
    inv = [row_mat[size:] for row_mat in aug_mat]
    
    return inv


In [53]:
# Create a class that finds the inverse of a 3x3 matrix using cofactors method 
class CofactorInv3:
    def __init__(self, matrix) -> None:
        self.mat = matrix 
    
    def transpose(self, matrix):
        """ 
            This method transposes a matrix

            Arguments
            ---------
            matrix: matrix to be transposed (list-of-lists)

            Returns
            --------
            trans: transpose of matrix
        """
        trans = [[matrix[j][i] for j in range(len(matrix))] for i in range(len(matrix[0]))]
        return trans 

    def get_minor(self, matrix, row, col):
        """ 
            This method returns the minor of a matrix

            Arguments
            ---------
            matrix: matrix (list-of-lists)
            row: row index
            col: column index

            Returns
            --------
            minor: minor of a 2x2 matrix
        """
        # chop off the row and column of the element
        temp_mat = copy.deepcopy(matrix)


        
        #TODO: Write your code to get the minor of a matrix
        
        # create a zero  array of the same size as the matrix
        # minor = np.zeros(matrix.shape)
        
        del temp_mat[row]
        # minor = [r.pop(col) for r in minor]
        minor = []
        for column in temp_mat:
            del column[col]
            minor.append(column)
        
        return minor # uncomment this when you are done

    
    def det(self, matrix):
        """ 
            This method calculates the determinant of a matrix 

            Arguments
            ---------
            matrix: matrix (list-of-lists)

            Returns
            --------
            determinant: determinant of matrix
        """
        # base case: if matrix is 2x2
        
        if len(matrix) == 2:
            return matrix[0][0] * matrix[1][1] - matrix[0][1] * matrix[1][0]
        
        # if matrix is 3x3
        if len(matrix) != len(matrix[0]):
            raise ValueError("Matrix is not square! it has no determinant")
        determinant = 0
        for row in range(len(matrix)):
            for col in range(len(matrix)):
                minor = self.get_minor(matrix, col, row)
                determinant +=( (-1) ** ( row) )* matrix[col][row] * self.det(minor)


            #TODOL Write your code to calculate the determinant of a 3X3 matrix
            # pass 
        return determinant # uncomment this when you are done

    def get_cofactor(self, matrix):
        """ 
            This method calculates the cofactor of a matrix 

            Arguments
            ---------
            matrix: matrix (list-of-lists)

            Returns
            --------
            cofactor: cofactor of matrix
        """
        cofactor = [] # initialize the cofactor matrix
        
        #TODO: Write your code to calculate the cofactor of a matrix
        for i in range(len(matrix)):
            row = []
            for j in range(len(matrix)):
                temp_matrix = self.get_minor(matrix, i, j)
                row.append((-1)**(i+j)*self.det(temp_matrix))
            cofactor.append(row)
            # pass

        
        return cofactor # uncomment this when you are done

    
    def get_inverse(self):
        """ 
            This method calculates the inverse of a matrix 

            Arguments
            ---------
            matrix: matrix (list-of-lists)

            Returns
            --------
            inv: inverse of matrix (adjoint/determinant where 
            adjoint is the transpose of the cofactor matrix)
        """
        # calculate the determinant of the matrix
        det = self.det(self.mat)

        if det == 0:
            raise ValueError("The matrix has no inverse")
        
        # calculate the cofactor of the matrix
        cofactor = self.get_cofactor(self.mat)  #None # TODO: Write your code to calculate the cofactor of a matrix
        # print(cofactor)
        
        # calculate the adjoint of the matrix
        adjoint = self.transpose(cofactor)

        # calculate the inverse of the matrix
        inv = [[element / det for element in row] for row in adjoint]
        
        return inv # uncomment this when you are done


## Verify the Gauss-Jordan and Cofactors Method

In [54]:
# verify your gauss-jordan inverse function
mat = [[0, 2, 0], [2, 2, 3], [4, -3, 0]]
inverse = inv_gauss_jordan(mat)
print(inverse)

[[0.375, 0.0, 0.25], [0.5, 0.0, 0.0], [-0.5833333333333334, 0.3333333333333333, -0.16666666666666666]]


In [55]:
# verify your Cofactors class
mat = [[0, 2, 0], [2, 2, 3], [4, -3, 0]]
cofac_obj = CofactorInv3(mat)
inverse = cofac_obj.get_inverse()
print(inverse)

[[0.375, 0.0, 0.25], [0.5, 0.0, 0.0], [-0.5833333333333334, 0.3333333333333333, -0.16666666666666666]]


## Verify Relationship Between Grayscale and RGB  (Using Quantized Grayscale Values)

In [56]:
# verify the relationship between grayscale and RGB for Cofactors (Using Quantized Grayscale values)
cofac_obj = CofactorInv3(rgb_matrix)
inverse = cofac_obj.get_inverse()

# Perform matrix multiplication
result = [0, 0, 0]
for i in range(3):
    for j in range(3):
        # TODO: Complete your matrix multiplication step (just one line of code)
        # pass
        result[i] += inverse[i][j] * gray_quant_matrix[j]

# Print the result
print(result)


[0.5942028985507477, 0.20289855072462615, 0.10144927536231307]


In [57]:
# verify the relationship between grayscale and RGB for Gauss-Jordan (Using Quantized Grayscale values)
inverse = inv_gauss_jordan(rgb_matrix)

# Perform matrix multiplication
result = [0, 0, 0]
for i in range(3):
    for j in range(3):
        # TODO: Complete your matrix multiplication step (just one line of code)
        pass
        result[i] += inverse[i][j] * gray_quant_matrix[j]

# Print the result
print(result)


[0.5942028985506624, 0.202898550724683, 0.10144927536232728]


## Verify Relationship Between Grayscale and RGB (Using Unquantized Grayscale Values)

In [58]:
## Verify Relationship Between Grayscale and RGB for Cofactors (Using Unquantized Grayscale Values)
cofac_obj = CofactorInv3(rgb_matrix)
inverse = cofac_obj.get_inverse()

# Perform matrix multiplication
result = [0, 0, 0]
for i in range(3):
    for j in range(3):
        # TODO: Complete your matrix multiplication step (just one line of code)
        # pass
        result[i] += inverse[i][j] * gray_matrix[j]
    

# Print the result
print(result)


[0.11400792218637434, 0.5869932312896253, 0.2989974091018581]


In [59]:
## Verify Relationship Between Grayscale and RGB for Gauss-Jordan (Using Unquantized Grayscale Values)
inverse = inv_gauss_jordan(rgb_matrix)

# Perform matrix multiplication
result = [0, 0, 0]
for i in range(3):
    for j in range(3):
        # TODO: Complete your matrix multiplication step (just one line of code)
        # pass
        result[i] += inverse[i][j] * gray_matrix[j]

# Print the result
print(result)


[0.1140079221863175, 0.5869932312896822, 0.2989974091018581]


### Convert RGB to Grayscale using Results from Gauss-Jordan or Cofactors Method

In [60]:
# convert rgb_matrix (list-of-lists) to grayscale

gray_conv = [[None for _ in range(799)] for _ in range(533)] # list to hold grayscale values 

for row in range(533):
    for col in range(799):
        r, g, b = col_img[row][col] # extract RGB values
        gray_val = (r*result[0] + g*result[1] + b*result[2])    #TODO: Put your equation to get the grayscale values here
        gray_conv[row][col] = gray_val


# save your result to a json file
with open('result/result_gray.json', 'w') as outfile:
    json.dump(gray_conv, outfile)


In [61]:
# view your image (if you are on a Linux machine, replace python below with python3)
# otherwise, running the code below should work
!python viewer.py --gray=result/result_gray.json